<a href="https://colab.research.google.com/github/hoky1227/bert_based-recommendation/blob/main/Transformer_berttokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install pytorch-transformers

     |████████████████████████████████| 2.0MB 18.2MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 3.2MB 48.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4939c8e329788e230e295c079a592f64e510fb256ba1a603c0893641b5ee52d4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 18.2MB/s 
     |████████████████████████████████| 1.2MB 45.9MB/s 
     |████████████████████████████████| 133kB 53.6MB/s 
     |████████████████████████████████| 7.3MB 53.0MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
ERROR: botocore 1.20.33 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
# % pip install sentencepiece
# !pip install tokenizers

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import tensorflow as tf
import torch
import tensorflow_datasets as tfds

from transformers import BertTokenizer
from transformers import BertForNextSentencePrediction, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
# import torch
from torch.utils.data import Dataset
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

import pandas as pd
import numpy as np
import random
import time
import datetime
import csv

In [ ]:
data_bert = pd.read_csv('/content/drive/MyDrive/train_bert.csv')
# data_space = pd.read_csv('/content/drive/MyDrive/train_space.csv')
Detail = pd.read_csv('/content/drive/MyDrive/Detail.csv')
data_bert.pop('Unnamed: 0')
# data_space.pop('Unnamed: 0')
Detail.pop('Unnamed: 0')
data_bert = data_bert.sample(frac=1).reset_index(drop=True)
data_bert.head()

,Customer,Detail,label
0,35061.0,REGENCY CAKESTAND 3 TIER,RED SPOTS WRAPREGENCY CAKESTAND 3 TIER
1,33983.0,HANGING JAM JAR T-LIGHT HOLDER&&MAGIC DRAWING ...,BALLOON PUMP WITH 10 BALLOONS
2,33050.0,PEARL CRYSTAL PUMPKIN T-LIGHT HLDR&&HANGING SP...,TOY TIDY PINK RETROSPOT
3,36918.0,PLASTERS IN TIN SKULLS&&WHITE HANGING HEART T-...,PLASMATRONIC LAMP
4,33223.0,CAMOFLAGE DESIGN TEDDY,KINGS CHOICE MUG


In [ ]:
print(len(data_bert))
# print(len(data_space))

11575


In [ ]:
label_bert = data_bert['label'].values.tolist()
# label_space = data_space['label'].values.tolist()
print(len(label_bert))
# print(len(label_space))

11575


In [ ]:
detail = Detail['0'].unique()
detail = detail.tolist()
len(detail)

3487

In [ ]:
dic = {string : i for i,string in enumerate(detail)}
print(dic)
len(dic)

{'RED SPOTTY CHILDS UMBRELLA': 0, 'EDWARDIAN PARASOL RED': 1, 'EDWARDIAN PARASOL NATURAL': 2, 'EDWARDIAN PARASOL BLACK': 3, 'EDWARDIAN PARASOL PINK': 4, 'DOORMAT SPOTTY HOME SWEET HOME': 5, 'DOORMAT WELCOME TO OUR HOME': 6, 'DOORMAT I LOVE LONDON': 7, 'DOORMAT CHRISTMAS VILLAGE': 8, 'DOORMAT BLACK FLOCK': 9, 'DOORMAT WELCOME SUNRISE': 10, 'DOORMAT 3 SMILEY CATS': 11, 'DOORMAT HOME SWEET HOME BLUE': 12, 'DOORMAT FANCY FONT HOME SWEET HOME': 13, 'DOORMAT UNION FLAG': 14, 'DOORMAT MERRY CHRISTMAS RED': 15, 'DOORMAT RESPECTABLE HOUSE': 16, 'DOORMAT FAIRY CAKE': 17, 'DOORMAT PEACE ON EARTH BLUE': 18, 'DOORMAT WELCOME PUPPIES': 19, 'DOORMAT NEW ENGLAND': 20, 'DOORMAT AIRMAIL': 21, 'DOORMAT RED SPOT': 22, 'DOORMAT UNION JACK GUNS AND ROSES': 23, 'WRAP PINK FAIRY CAKES': 24, 'VINTAGE CARAVAN GIFT WRAP': 25, 'BIRTHDAY BANQUET GIFT WRAP': 26, 'RED SPOTS  WRAP': 27, 'RAIN PONCHO WITH RED SPOTS': 28, 'STRAWBERRY CERAMIC TRINKET BOX': 29, 'SWEETHEART CERAMIC TRINKET BOX': 30, 'LOVE HEART SOCK HANGE

3487

In [ ]:
labels = []
for v in (label_bert):
    labels.append(dic.get(v))
print(labels)
print(len(labels))

[None, 187, 256, 2053, 777, 587, None, 297, None, 38, 551, 649, 504, 208, 180, 1195, 1617, 682, 310, 661, 484, 3285, 2553, 1592, 2078, 296, 570, 392, None, 139, None, 1195, 1201, 214, 629, 1546, 2373, 610, 247, 2505, 115, 49, 946, 1431, 308, 1751, 300, 764, 1623, 2305, 657, 1321, 439, 1376, 1062, 374, 132, 749, 23, 475, 649, 1273, 559, 267, 177, 424, 79, 2142, 1869, 1687, 1991, 46, 419, 2096, 2103, 1066, 2740, 1516, 644, 313, 1260, 139, 418, 177, 643, 360, 1528, 64, 2219, 867, 1519, None, 949, 332, 178, 1066, 64, 2684, 971, 1049, 946, 1526, 486, 1242, 483, 2628, 937, 590, 617, 400, 1612, 988, 1020, 679, 299, 1419, 661, None, 1937, 988, 290, 1878, 1295, 667, 258, 241, 195, 278, 372, 50, 619, 684, 781, None, 950, 190, 411, 684, 130, 2258, 3151, 1814, 139, 170, 300, 1941, 2096, 357, 196, 649, 823, 989, 377, 136, 190, 453, 286, 1301, 1487, 1765, 1442, 1269, 808, 941, 1105, None, 265, 452, 37, 539, 649, 941, 124, 3277, 2404, 1047, 562, 459, 1560, 1981, 502, 1723, 1434, 0, 2147, 590, 215, 66

In [ ]:
labelsdf = pd.DataFrame(labels)
data1 = pd.concat([data_bert, labelsdf], axis=1)
data1.dropna(inplace=True)
len(data1)

11259

In [ ]:
data1.head()

,Customer,Detail,label,0
1,33983.0,HANGING JAM JAR T-LIGHT HOLDER&&MAGIC DRAWING ...,BALLOON PUMP WITH 10 BALLOONS,187.0
2,33050.0,PEARL CRYSTAL PUMPKIN T-LIGHT HLDR&&HANGING SP...,TOY TIDY PINK RETROSPOT,256.0
3,36918.0,PLASTERS IN TIN SKULLS&&WHITE HANGING HEART T-...,PLASMATRONIC LAMP,2053.0
4,33223.0,CAMOFLAGE DESIGN TEDDY,KINGS CHOICE MUG,777.0
5,37092.0,PARTY METAL SIGN&&STRAWBERRY CERAMIC TRINKET B...,RED HEART SHAPE LOVE BUCKET,587.0


In [ ]:
labels = data1[0].values
labels = labels.tolist()
len(labels)

11259

In [ ]:
print(labels[:5])

[187.0, 256.0, 2053.0, 777.0, 587.0]


In [ ]:
cut = round(len(data1) * 0.7)
train, test = data1[:cut], data1[cut:]
train_label, test_label = data1[:cut][0], data1[cut:][0]
train_label, test_label = train_label.to_list(), test_label.to_list()

In [ ]:
print(train_label)

[187.0, 256.0, 2053.0, 777.0, 587.0, 297.0, 38.0, 551.0, 649.0, 504.0, 208.0, 180.0, 1195.0, 1617.0, 682.0, 310.0, 661.0, 484.0, 3285.0, 2553.0, 1592.0, 2078.0, 296.0, 570.0, 392.0, 139.0, 1195.0, 1201.0, 214.0, 629.0, 1546.0, 2373.0, 610.0, 247.0, 2505.0, 115.0, 49.0, 946.0, 1431.0, 308.0, 1751.0, 300.0, 764.0, 1623.0, 2305.0, 657.0, 1321.0, 439.0, 1376.0, 1062.0, 374.0, 132.0, 749.0, 23.0, 475.0, 649.0, 1273.0, 559.0, 267.0, 177.0, 424.0, 79.0, 2142.0, 1869.0, 1687.0, 1991.0, 46.0, 419.0, 2096.0, 2103.0, 1066.0, 2740.0, 1516.0, 644.0, 313.0, 1260.0, 139.0, 418.0, 177.0, 643.0, 360.0, 1528.0, 64.0, 2219.0, 867.0, 1519.0, 949.0, 332.0, 178.0, 1066.0, 64.0, 2684.0, 971.0, 1049.0, 946.0, 1526.0, 486.0, 1242.0, 483.0, 2628.0, 937.0, 590.0, 617.0, 400.0, 1612.0, 988.0, 1020.0, 679.0, 299.0, 1419.0, 661.0, 1937.0, 988.0, 290.0, 1878.0, 1295.0, 667.0, 258.0, 241.0, 195.0, 278.0, 372.0, 50.0, 619.0, 684.0, 781.0, 950.0, 190.0, 411.0, 684.0, 130.0, 2258.0, 3151.0, 1814.0, 139.0, 170.0, 300.0, 

In [ ]:
# import tensorflow_hub as hub
# FullTokenizer = bert.bert_tokenization.FullTokenizer
# bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)

In [ ]:
train['Detail'] = train['Detail'].str.replace('&&', ' [SEP] ')
sentences = train['Detail']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'[CLS] PACK OF 12 LONDON TISSUES [SEP] PACK OF 12 WOODLAND TISSUES [SEP] HANGING JAM JAR T-LIGHT HOLDER [SEP] PLASTERS IN TIN CIRCUS PARADE [SEP] GROW YOUR OWN PLANT IN A CAN [SEP] PLASTERS IN TIN SPACEBOY [SEP] BABUSHKA LIGHTS STRING OF 10 [SEP] SET/3 RUSSIAN DOLL STACKING TINS [SEP] SET 10 LIGHTS NIGHT OWL [SEP] ICE CREAM DESIGN GARDEN PARASOL [SEP] PINK PICNIC BARBECUE BUCKET [SEP]'

In [ ]:
# # BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# tokenized_texts_list = train['Detail'].to_list()
# tokenized_texts = []
# for v in tokenized_texts_list:
    # tokenized_texts.append([v])
print (sentences[0])
print (tokenized_texts[0])
len(tokenized_texts)

100%|██████████| 231508/231508 [00:00<00:00, 19402448.36B/s]


[CLS] HANGING JAM JAR T-LIGHT HOLDER [SEP] MAGIC DRAWING SLATE SPACEBOY [SEP] PICTURE DOMINOES [SEP] BLACK RECORD COVER FRAME [SEP] MAGIC DRAWING SLATE CIRCUS PARADE [SEP] PLASTERS IN TIN STRONGMAN [SEP] PLASTERS IN TIN SPACEBOY [SEP] ROUND SNACK BOXES SET OF 4 FRUITS [SEP] DOOR MAT RED SPOT [SEP] CHILDS BREAKFAST SET CIRCUS PARADE [SEP] PLASTERS IN TIN CIRCUS PARADEPACK OF 12 TRADITIONAL CRAYONS [SEP] SET 12 RETRO WHITE CHALK STICKS [SEP] SET 12 KIDS COLOUR  CHALK STICKS [SEP] TRADITIONAL MODELLING CLAY [SEP] PACK OF 12 COLOURED PENCILS [SEP] SET OF 72 RETRO SPOT PAPER  DOILIES [SEP] SET OF 9 HEART SHAPED BALLOONS [SEP] PACK OF 6 HANDBAG GIFT BOXES [SEP] PACK OF 6 SWEETIE GIFT BOXES [SEP] SET/5 RED SPOTTY LID GLASS BOWLS [SEP] GLASS JAR PEACOCK BATH SALTS [SEP] GLASS JAR DAISY FRESH COTTON WOOL [SEP] BOX OF VINTAGE JIGSAW BLOCKS [SEP]
['[CLS]', 'hanging', 'jam', 'jar', 't', '-', 'light', 'holder', '[SEP]', 'magic', 'drawing', 'slate', 'space', '##boy', '[SEP]', 'picture', 'domino', '#

7881

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 512

In [ ]:
# tok = []
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         tok.append(len(tokenized_texts[i]))
# print(max(tok))
# print(len(tok))
# print(len(tokenized_texts))

In [ ]:
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         while 1:
#             id = tokenized_texts[i].index('[SEP]')
#             tokenized_texts[i] = tokenized_texts[i][id:]
#             tokenized_texts[i][0] = '[CLS]'
#             if len(tokenized_texts[i]) <= 512:
#                 break
# tok = []
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         tok.append(len(tokenized_texts[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts))

In [ ]:
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        while 1:
            id = tokenized_texts[i].index('[SEP]')
            tokenized_texts[i] = tokenized_texts[i][id:]
            tokenized_texts[i][0] = '[CLS]'
            if len(tokenized_texts[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        tok.append(len(tokenized_texts[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts))

0
7881


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
# MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

print(input_ids[0])
len(input_ids)

[  101  5689  9389 15723  1056  1011  2422  9111   102  3894  5059 12796
  2686 11097   102  3861 23968  2229   102  2304  2501  3104  4853   102
  3894  5059 12796  9661  7700   102 15673  2015  1999  9543  2844  2386
   102 15673  2015  1999  9543  2686 11097   102  2461 19782  8378  2275
  1997  1018 10962   102  2341 13523  2417  3962   102  2775  2015  6350
  2275  9661  7700   102 15673  2015  1999  9543  9661  7700 23947  1997
  2260  3151 13675 28852  3619   102  2275  2260 22307  2317 16833 12668
   102  2275  2260  4268  6120 16833 12668   102  3151 19518  5726   102
  5308  1997  2260 11401 14745  2015   102  2275  1997  5824 22307  3962
  3259  9193 11983   102  2275  1997  1023  2540  5044 22163   102  5308
  1997  1020  2192 16078  5592  8378   102  5308  1997  1020 22872  5592
  8378   102  2275  1013  1019  2417  3962  3723 11876  3221 15220   102
  3221 15723 18931  7198 23480   102  3221 15723 10409  4840  6557 12121
   102  3482  1997 13528 10147  5620 10376  5991   

7881

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    train_label, 
                                                                                    # random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                    #    random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  2540,  1997, 15536,  9102,  2312,   102,  2540,  1997, 15536,
         9102,  2235,   102,  3500, 11703,  1010,  5689, 14556,  2665,   102,
        11446,  1010,  5689,  2540,  1009,  5259,   102,  2540, 11554, 29461,
        21711,  2235,   102, 10882,  3669, 16523,  2483,  2540,  2007,  9112,
          102,  2540, 11554, 29461, 21711,  2312,   102,  2540, 10882,  3669,
        28637, 10855,  2235,   102,  2540, 10882,  3669, 28637, 10855,  2312,
          102, 10720,  2072,  4597,   102,  2176,  8103,  2317,  2293, 12887,
          102,  2317,  5689,  2540,  1056,  1011,  2422,  9111,   102, 14405,
         2317,  7318,  2540, 12313,   102,  2317, 12074,  2015,  5239,  1018,
        22497,   102,  3019, 11286, 13541,  2235,   102,  3019, 11286, 13541,
         2312,   102,  6949, 12074,  3661, 14513,   102, 12074,  3145,  5239,
          102,  6949, 12074,  7163,  3108,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
# 배치 사이즈
batch_size = 2

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
test['Detail'] = test['Detail'].str.replace('&&', ' [SEP] ')
sentences = test['Detail']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['[CLS] DOOR MAT 3 SMILEY CATS [SEP] WHITE HANGING HEART T-LIGHT HOLDER [SEP]',
 '[CLS] SET OF 3 COLOURED  FLYING DUCKS [SEP] RECORD FRAME 7" SINGLE SIZE [SEP] WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGING HEART T-LIGHT HOLDER [SEP] RED WOOLLY HOTTIE WHITE HEART. [SEP] RECORD FRAME 7" SINGLE SIZE [SEP] LUNCH BAG RED SPOTTY [SEP] STRAWBERRY CERAMIC TRINKET BOX [SEP] PINK MARSHMALLOW SCARF KNITTING KIT [SEP] VINTAGE UNION JACK PENNANT [SEP] METAL SIGN NEIGHBOURHOOD WITCH [SEP] VINTAGE UNION JACK CUSHION COVER [SEP] METAL SIGN EMPIRE TEA [SEP]',
 '[CLS] RED SPOTTY PEG BAG [SEP] LUNCH BAG  BLACK SKULL. [SEP] WASH BAG VINTAGE ROSE PAISLEY [SEP] BLUE SQUARE COMPACT MIRROR [SEP] CLOTHES PEGS RETROSPOT PACK 24 [SEP] BIRDHOUSE DECORATION MAGIC GARDEN [SEP] LUNCH BAG RED SPOTTY [SEP] GREEN SQUARE COMPACT MIRROR [SEP] WOODEN PICTURE FRAME WHITE FINISH [SEP] LUNCH BAG SPACEBOY DESIGN [SEP] CHERRY BLOSSOM BATHROOM CABINET [SEP] SCANDINAVIAN PAISLEY PICNIC BAG [SEP] PINK VINTAGE PAISLEY PICNIC BAG

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] DOOR MAT 3 SMILEY CATS [SEP] WHITE HANGING HEART T-LIGHT HOLDER [SEP]
['[CLS]', 'door', 'mat', '3', 'smiley', 'cats', '[SEP]', 'white', 'hanging', 'heart', 't', '-', 'light', 'holder', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
# MAX_LEN = 128

In [ ]:
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        while 1:
            id = tokenized_texts[i].index('[SEP]')
            tokenized_texts[i] = tokenized_texts[i][id:]
            tokenized_texts[i][0] = '[CLS]'
            if len(tokenized_texts[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        tok.append(len(tokenized_texts[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts))

0
3378


In [ ]:
# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101,  2341, 13523,  1017, 27420,  8870,   102,  2317,  5689,
        2540,  1056,  1011,  2422,  9111,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(test_label)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101,  2341, 13523,  1017, 27420,  8870,   102,  2317,  5689,  2540,
         1056,  1011,  2422,  9111,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
# 배치 사이즈
# batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(detail))
model.cuda()

100%|██████████| 440473133/440473133 [00:07<00:00, 58981416.87B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()

    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def intop10(logit, label, mode=0, top=10):
    '''mode = 0 (default) if you want to see boolean return'''
    l = []
    for _ in range(top):
        pred = np.argmax(logit)
        l.append(pred)
        logit[0][pred] = np.min(logit)
    if mode == 0:
        print(l)
        if label in l:
            return True
        else: return False
    elif mode == 1:
        if label in l:
            return 1
        else: return 0
    elif mode == 2:
        print(l)
        return l

In [ ]:
def cal_acc(preds, label):
    acc = 0
    for v in preds:
        acc += intop10(v, label, mode=1)
    return acc / len(preds)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)



# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        b_labels = b_labels.type(torch.LongTensor).to(device)
        # print(b_labels.type())
        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    acc = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        # acc = cal_acc(logits, label_ids)
    print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    # print("  Accuracy: {0:.3f}".format(acc))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


KeyboardInterrupt: ignored

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
acc = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    # acc = cal_acc(logits, label_ids)
# print("  Accuracy: {0:.3f}".format(acc)) # eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,689.    Elapsed: 0:00:07.
  Batch   200  of  1,689.    Elapsed: 0:00:15.
  Batch   300  of  1,689.    Elapsed: 0:00:22.
  Batch   400  of  1,689.    Elapsed: 0:00:29.
  Batch   500  of  1,689.    Elapsed: 0:00:37.
  Batch   600  of  1,689.    Elapsed: 0:00:44.
  Batch   700  of  1,689.    Elapsed: 0:00:52.
  Batch   800  of  1,689.    Elapsed: 0:00:59.
  Batch   900  of  1,689.    Elapsed: 0:01:07.
  Batch 1,000  of  1,689.    Elapsed: 0:01:15.
  Batch 1,100  of  1,689.    Elapsed: 0:01:22.
  Batch 1,200  of  1,689.    Elapsed: 0:01:30.
  Batch 1,300  of  1,689.    Elapsed: 0:01:38.
  Batch 1,400  of  1,689.    Elapsed: 0:01:46.
  Batch 1,500  of  1,689.    Elapsed: 0:01:54.
  Batch 1,600  of  1,689.    Elapsed: 0:02:02.

Accuracy: 0.016
Test took: 0:02:09


In [ ]:
torch.save(model, '/content/drive/MyDrive/BERTmodel_bert.pt')

In [ ]:
# torch.save(model.state_dict(), './BERTmodel_stat.pt')

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512
    # if len(tokenized_texts) > MAX_LEN:
    #     tokenized_texts = tokenized_texts[-Max_Len::]    
    for i in range(len(tokenized_texts)):
        if len(tokenized_texts[i]) > MAX_LEN:
            while 1:
                id = tokenized_texts[i].index('[SEP]')
                tokenized_texts[i] = tokenized_texts[i][id:]
                tokenized_texts[i][0] = '[CLS]'
                if len(tokenized_texts[i]) <= MAX_LEN:
                    break
    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
# def kv(dic, val):
#     return [k for k, v in dic.items() if v == val]

prod_dic = {v:k for k,v in dic.items()}
print(prod_dic)

def findprod(v):
    p = prod_dic.get(v)
    return p

In [ ]:
test.reset_index(inplace=True)
test.pop('index')

0        8108
1        8109
2        8110
3        8111
4        8112
        ...  
3373    11570
3374    11571
3375    11572
3376    11573
3377    11574
Name: index, Length: 3378, dtype: int64

In [ ]:
test.head()

,Customer,Detail,label,0
0,37681.0,DOOR MAT 3 SMILEY CATS [SEP] WHITE HANGING HEA...,STRAWBERRY SCENTED SET/9 T-LIGHTS,1013.0
1,33777.0,SET OF 3 COLOURED FLYING DUCKS [SEP] RECORD F...,RED SPOTTY CHILDS APRON,288.0
2,37850.0,RED SPOTTY PEG BAG [SEP] LUNCH BAG BLACK SKUL...,WOODEN FRAME ANTIQUE WHITE,947.0
3,36296.0,MINI WOODEN HAPPY BIRTHDAY GARLAND [SEP] PLACE...,RED SPOTTY OVEN GLOVE DOUBLE,1249.0
4,33080.0,SET OF 6 VINTAGE NOTELETS KIT [SEP] SILK PURSE...,SILK PURSE RUSSIAN DOLL PINK,557.0


In [ ]:
model = torch.load('/content/drive/MyDrive/BERTmodel_bert.pt')

In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=5)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.031


In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.054


In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=15)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.069


In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=20)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.082


In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=25)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.094


In [ ]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=30)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.109


In [ ]:
table = test.copy()
for i in range(len(table)):
    logits = test_sentences([tabe['Detail'][i]])
    table['preds'][i] = intop10(logits, test[0][i], mode=2)
 

In [ ]:
print("Hits: {}".format(acc))

Hits: 183


In [ ]:
logits = test_sentences([data1['Detail'][6]])

print(logits)
print((np.argmax(logits)))
print(len(logits[0]))
print(data1[0][6])

[[ -1.0715597  -0.5765179  -0.0243252 ... -10.531099  -10.016944
  -10.483797 ]]
661
3487
2429.0


In [ ]:
logits = test_sentences([test['Detail'][100]])

print(logits)
print(np.argmax(logits))
# print(len(logits))
print(test['label'][100])
intop10(logits, test['label'][100], mode=1)

[[-2.7736073  -1.2774974  -0.61112374 ... -8.012796   -8.468978
  -8.195465  ]]
661
MULTI COLOUR SILVER T-LIGHT HOLDER


0

In [ ]:
logits = test_sentences([data1['Detail'][100]])

# print(logits)
print(np.argmax(logits))
# print(len(logits))
print(data1[0][100])

661
439.0


In [ ]:
for i in range(0, len(test), 500):
    logits = test_sentences([test['Detail'][i]])
    # print(np.argmax(logits))
    print(intop10(logits, test[0][i], mode=0))

[661, 400, 617, 914, 195, 215, 189, 121, 649, 190]
False
[661, 400, 792, 947, 649, 617, 130, 195, 256, 215]
False
[661, 400, 792, 617, 947, 215, 190, 256, 195, 914]
False
[661, 400, 947, 617, 590, 130, 792, 195, 306, 649]
False
[661, 400, 947, 617, 195, 792, 590, 121, 649, 306]
False
[661, 649, 400, 1435, 829, 782, 947, 29, 130, 989]
False
[661, 400, 617, 914, 215, 792, 947, 256, 190, 195]
False


In [ ]:
print(findprod(661))
print(findprod(400))

WHITE HANGING HEART T-LIGHT HOLDER
DOOR MAT UNION FLAG


In [ ]:
a = test['Detail'][199]
logits = test_sentences([a])
print('sent:',a)
b = intop10(logits, test[0][199], mode=2)
print('label:', findprod(test[0][199]), test[0][199])
p = []
for v in b:
    p.append(findprod(v))
print(p)

sent: GLASS JAR ENGLISH CONFECTIONERY [SEP] GLASS JAR DAISY FRESH COTTON WOOL [SEP] SLEEPING CAT ERASERS [SEP] DECORATION, HANGING HEART+MIRROR [SEP] SEWING SUSAN 21 NEEDLE SET [SEP] WOODLAND  STICKERS [SEP] 12 IVORY ROSE PEG PLACE SETTINGS [SEP] COLOURING PENCILS BROWN TUBE [SEP] GLASS JAR ENGLISH CONFECTIONERY [SEP] MOCK LOBSTER FRIDGE MAGNET [SEP] WHITE HANGING HEART T-LIGHT HOLDER [SEP] GARLAND WOODEN HAPPY EASTER [SEP] EASTER TIN BUCKET [SEP] SET OF 4 ENGLISH ROSE COASTERS [SEP] RED SPOTTY CUP [SEP] BLUE  SPOTTY CUP [SEP] SET 12 RETRO WHITE CHALK STICKS [SEP] GREEN  SPOTTY PLATE [SEP] ALPHABET HEARTS STICKER SHEET [SEP] BIRD DECORATION RED SPOT [SEP] BIRD DECORATION GREEN  SPOT [SEP] GREEN SPOTTY CUP [SEP] PACK 20 ENGLISH ROSE PAPER NAPKINS [SEP] PAPER BUNTING WHITE LACE [SEP] EMBOSSED GLASS TEALIGHT HOLDER [SEP] WOODLAND ANIMAL  WRITING SET [SEP] S/4 PINK FLOWER CANDLES IN BOWL [SEP] CERAMIC CAKE DESIGN SPOTTED PLATE [SEP] TEA TIME TABLE CLOTH [SEP] OCEAN STRIPE HAMMOCK [SEP] DOO

In [ ]:
data1['label'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    157
DOOR MAT UNION FLAG                    64
REGENCY CAKESTAND 3 TIER               53
HOME BUILDING BLOCK WORD               51
PARTY BUNTING                          50
                                     ... 
PINK HAWAIIAN PICNIC HAMPER FOR 2       1
IVORY HANGING DECORATION  HEART         1
VINTAGE ROSE BEAD BRACELET BLUE         1
RETRO PINK BALL ASHTRAY                 1
LETTER "N" BLING KEY RING               1
Name: label, Length: 1818, dtype: int64

In [ ]:
data1[0].value_counts()

661.0     157
400.0      64
130.0      53
121.0      51
649.0      50
         ... 
2147.0      1
3145.0      1
3278.0      1
1075.0      1
659.0       1
Name: 0, Length: 1818, dtype: int64

모델 참고 링크

https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

https://huggingface.co/transformers/model_doc/bert.html

https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=VJ76KiP_dLn-&uniqifier=1

In [ ]:
# class NsmcDataset(Dataset):
#     ''' Naver Sentiment Movie Corpus Dataset '''
#     def __init__(self, df):
#         self.df = df

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         text = self.df.iloc[idx, 0]
#         label = self.df.iloc[idx, 1]
#         return text, label

In [ ]:
# la = pd.DataFrame(labels)
# display(la.head())
# train_data = pd.concat([train, la], axis=1)
# train_data.head()

In [ ]:
# nsmc_train_dataset = NsmcDataset(train_data)
# train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
# device = torch.device("cuda")
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(detail))
# model.to(device)

BERT

In [ ]:
# !git clone https://github.com/google-research/bert.git

In [ ]:
# !python /content/bert/create_pretraining_data.py \
#   --input_file=train \
#   --output_file=/tmp/tf_examples.tfrecord \
#   --vocab_file=$BERT_BASE_DIR/vocab.txt \
#   --do_lower_case=False \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=5

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# optimizer = Adam(model.parameters(), lr=1e-6)

# itr = 1
# p_itr = 500
# epochs = 1
# total_loss = 0
# total_len = 0
# total_correct = 0


# model.train()
# for epoch in range(epochs):
    
#     for text, label in train_loader:
#         optimizer.zero_grad()
        
#         # encoding and zero padding
#         # encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
#         encoded_list = tokenized_texts
#         # padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
#         # 토큰을 숫자 인덱스로 변환
#         input_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_list]

#         # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
#         padded_list = pad_sequences(input_ids, maxlen=256, dtype="long", truncating="post", padding="post")
#         # print(padded_list[0])
#         # print(len(padded_list[0]))
#         # print(len(padded_list))
#         # print(type(padded_list))
#         sample = torch.tensor(padded_list).to(torch.int32)
#         sample, label = sample.to(device), label.to(device)
#         labels = torch.tensor(label).to(torch.int32)
#         outputs = model(sample, labels=labels)
#         loss, logits = outputs

#         pred = torch.argmax(F.softmax(logits), dim=1)
#         correct = pred.eq(labels)
#         total_correct += correct.sum().item()
#         total_len += len(labels)
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
        
#         if itr % p_itr == 0:
#             print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
#             total_loss = 0
#             total_len = 0
#             total_correct = 0

#         itr+=1

In [ ]:
# prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
# next_sentence = "The sky is blue due to the shorter wavelength of blue light."
# encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

# outputs = model(**encoding, labels=torch.LongTensor([1]))
# logits = outputs.logits
# assert logits[0, 0] < logits[0, 1] # next sentence was random